# Retrieving raw JSON data for Oracle OpenWorld & CodeOne Session Catalog into local JSON Files

The session catalog data is available from a REST API whose root is https://events.rainfocus.com/api/search.

A typical call to this API uses Headers (rfwidgetid: 'KKA8rC3VuZo5clh8gX5Aq07XFonUTLyU',rfapiprofileid: 'uGiII5rYGOjoHXOZx0ch4r7f1KzFC0zd') and Form values to retrieve specific session information.

In [52]:
import requests
import pandas as pd
import json
import math 
url = "https://events.rainfocus.com/api/search" # rooturl for session catalog
#headers required in each API call
querystring = {"search.sessiontype":"1518466139979001dQkv"} #session type corresponds to BOF

payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"size\"\r\n\r\n50\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"type\"\r\n\r\nsession\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"from\"\r\n\r\n{0}\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--"

headers = {
    'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    'rfapiprofileid': "uGiII5rYGOjoHXOZx0ch4r7f1KzFC0zd",
    'rfwidgetid': "KKA8rC3VuZo5clh8gX5Aq07XFonUTLyU",
    'cache-control': "no-cache"
}
# OOW
OOW_rfapiprofileid ="K9HkkU5es180AVTifYUgYembIKJ15CMM"
OOW_rfwidgetid= "VEsNDADSTFH5azU4dH1QslO3lhpQTy4U"

# CodeOne
CodeOne_rfapiprofileid= "uGiII5rYGOjoHXOZx0ch4r7f1KzFC0zd"
CodeOne_rfwidgetid =  "KKA8rC3VuZo5clh8gX5Aq07XFonUTLyU"

Data should be collected for all session types. The list of session types is shown  below :

In [53]:
sessionTypes =  {'BOF': '1518466139979001dQkv'
    , 'BQS': 'bqs'
    , 'BUS': '1519240082595001EpMm'
    , 'CAS': 'casestudy'
    , 'DEV': '1522435540042001BxTD'
    , 'ESS': 'ess'
    , 'FLP': 'flp'
    , 'GEN': 'general'
    , 'HOL': 'hol'
    , 'HOM': 'hom'
    , 'IGN': 'ignite'
    , 'KEY': 'option_1508950285425'
    , 'MTE':'1523906206279002QAu9'
    , 'PKN': '1527614217434001RBfj'
    , 'PRO': '1518464344082003KVWZ'
    , 'PRM': '1518464344082002KM3k'
    , 'TRN': '1518464344082001KHky'
    , 'SIG': 'sig'
    , 'THT': 'ts'
    , 'TLD': '1537894888625001RriS'
    , 'TIP': '1517517756579001F3CR'
    , 'TUT': 'tutorial'
    , 'TRN': '1518464344082001KHky'
                }
                
print("Mapping of Session Type to header-code", sessionTypes)



Mapping of Session Type to header-code {'BOF': '1518466139979001dQkv', 'BQS': 'bqs', 'BUS': '1519240082595001EpMm', 'CAS': 'casestudy', 'DEV': '1522435540042001BxTD', 'ESS': 'ess', 'FLP': 'flp', 'GEN': 'general', 'HOL': 'hol', 'HOM': 'hom', 'IGN': 'ignite', 'KEY': 'option_1508950285425', 'MTE': '1523906206279002QAu9', 'PKN': '1527614217434001RBfj', 'PRO': '1518464344082003KVWZ', 'PRM': '1518464344082002KM3k', 'TRN': '1518464344082001KHky', 'SIG': 'sig', 'THT': 'ts', 'TLD': '1537894888625001RriS', 'TIP': '1517517756579001F3CR', 'TUT': 'tutorial'}


In [54]:
def loadSessionDataForSessionTypeStartingAt(key,sessionType, startingAt):
    querystring = {"search.sessiontype":sessionType} #session type corresponds to BOF
    
    response = requests.request("POST", url, data=payload.format(startingAt), headers=headers, params=querystring)
    return response.text

def loadSessionDataForSessionType(key,sessionType):
    tempDict = json.loads(loadSessionDataForSessionTypeStartingAt( key, sessionType,0))
    sl= tempDict ["sectionList"][0]
    sessions = sl["items"]
    total = sl.get("total",-1)
    received = sl['numItems']
    startingAt = sl['from']
    print(" ** Received from " , startingAt," set of ", received , "items, out of a total of ", total);
    if total > received:
        requestTotal = math.ceil(total/50)
        print ("number of additional requests = ",requestTotal-1)
        for k in range(1,requestTotal):
            # for subsequent request, the response does not contain the nested sectionList element
            tempDict = json.loads(loadSessionDataForSessionTypeStartingAt( key, sessionType, k*50))
            items = tempDict['items']

            received = tempDict['numItems']
            startingAt = tempDict['from']
            print(" ** Received from " , startingAt," set of ", received , "items, out of a total of ", total);
            #combine sessions with new set of items
            sessions = sessions + items
    #the JSON object is converted to a Dict; this Dict contains scalar values such as strings and more complex values such as Arrarys and nested Dicts)
    ss = pd.DataFrame(sessions)
    return ss;

In [55]:
#loop over all session types and invoke function loadSessionDataForSessionType
print("Loading CodeOne details")
headers['rfapiprofileid'] = CodeOne_rfapiprofileid
headers['rfwidgetid'] = CodeOne_rfwidgetid
for key, value in sessionTypes.items():
    ss = loadSessionDataForSessionType(key, value)
    ss.to_json("oow2018-sessions_codeone_{0}.json".format(key), force_ascii=False)

print("Loading OOW details")
headers['rfapiprofileid'] = OOW_rfapiprofileid
headers['rfwidgetid'] = OOW_rfwidgetid
for key, value in sessionTypes.items():
    ss = loadSessionDataForSessionType(key, value)
    ss.to_json("oow2018-sessions_oow_{0}.json".format(key), force_ascii=False)
    


Loading CodeOne details
 ** Received from  0  set of  43 items, out of a total of  43
 ** Received from  0  set of  0 items, out of a total of  0
 ** Received from  0  set of  7 items, out of a total of  7
 ** Received from  0  set of  7 items, out of a total of  7
 ** Received from  0  set of  50 items, out of a total of  398
number of additional requests =  7
 ** Received from  50  set of  50 items, out of a total of  398
 ** Received from  100  set of  50 items, out of a total of  398
 ** Received from  150  set of  50 items, out of a total of  398
 ** Received from  200  set of  50 items, out of a total of  398
 ** Received from  250  set of  50 items, out of a total of  398
 ** Received from  300  set of  50 items, out of a total of  398
 ** Received from  350  set of  48 items, out of a total of  398
 ** Received from  0  set of  0 items, out of a total of  0
 ** Received from  0  set of  0 items, out of a total of  0
 ** Received from  0  set of  1 items, out of a total of  1
 *

Below is the result of making a request for session details. The really interesting stuff is hidden in the first element of the sectionList array.

In [56]:
import pathlib

# define the path
currentDirectory = pathlib.Path('.')

for currentFile in currentDirectory.iterdir():  
    print(currentFile)

oow2018-sessions_codeone_PKN.json
oow2018-sessions_oow_ESS.json
oow2018-sessions_codeone_PRO.json
oow2018-sessions_oow_TIP.json
OOW2018 Session Catalog - retrieving raw session data in JSON files.ipynb
oow2018-sessions_oow_THT.json
.ipynb_checkpoints
oow2018-sessions-wrangled.json
oow2018-sessions_codeone_DEV.json
oow2018-sessions_codeone_BOF.json
oow2018-sessions_codeone_BUS.json
oow2018-sessions_codeone_MTE.json
oow2018-sessions_oow_PKN.json
oow2018-sessions_codeone_PRM.json
oow2018-sessions_codeone_TLD.json
oow2018-sessions_oow_PRM.json
oow2018-sessions_codeone_HOL.json
oow2018-sessions_codeone_KEY.json
oow2018-sessions_oow_TRN.json
oow2018-sessions_oow_HOM.json
oow2018-sessions_codeone_BQS.json
oow2018-sessions_oow_TLD.json
oow2018-sessions_codeone_TRN.json
oow2018-sessions_oow_KEY.json
oow2018-sessions_codeone_ESS.json
oow2018-sessions.json
oow2018-sessions_oow_CAS.json
oow2018-sessions_oow_FLP.json
oow2018-sessions_oow_BQS.json
OOW2018 Session Catalog - retrieving raw session dat

In [57]:
#as a test, try to load data from one of the generated files 
dubss = pd.read_json("oow2018-sessions_oow_HOL.json")
dubss

,abbreviation,abstract,allowDoubleBooking,attributevalues,code,codeParts,code_id,es_metadata_id,event,eventCode,...,type_displayorder,type_displayorder_string,useDoubleBooking,useWaitingList,videos,viewAccess,viewAccessPublic,viewFileAccess,waitlistAccess,waitlistLimit
0,HOL6306,"Enhance the security of your Oracle HCM, ERP, ...",0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6306,"{'alpha0': 'HOL', 'numeric1': '6306'}",hol6306,15311715163230013Hbl,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
1,HOL6286,Oracle Data Integration Platform Cloud allows ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6286,"{'alpha0': 'HOL', 'numeric1': '6286'}",hol6286,1530813370901001gB2W,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
10,HOL6325,In this session learn to develop and deploy a ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6325,"{'alpha0': 'HOL', 'numeric1': '6325'}",hol6325,15313473829130018nP4,Oracle OpenWorld,oow18,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
11,HOL6328,In this session learn about the latest feature...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6328,"{'alpha0': 'HOL', 'numeric1': '6328'}",hol6328,1531350426847001WHKn,Oracle OpenWorld,oow18,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
12,HOL6331,In this hands-on lab learn the basics of Oracl...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6331,"{'alpha0': 'HOL', 'numeric1': '6331'}",hol6331,1531436407190001W0ep,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
13,HOL6684,The size and scale of high-performance computi...,0,"[{'value': 'Intermediate', 'attributevalue_id'...",HOL6684,"{'alpha0': 'HOL', 'numeric1': '6684'}",hol6684,15372125646130014Ryp,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",3
14,HOL6334,In this lab learn how to use Vagrant to automa...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6334,"{'alpha0': 'HOL', 'numeric1': '6334'}",hol6334,1531850104620001VdEf,Oracle OpenWorld,oow18,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
15,HOL6305,Are you challenged with increasing user adopti...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6305,"{'alpha0': 'HOL', 'numeric1': '6305'}",hol6305,1531160180647001y4NT,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
16,HOL6309,Oracle Blockchain Cloud Service provides a pla...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6309,"{'alpha0': 'HOL', 'numeric1': '6309'}",hol6309,1531173049279001wPQp,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
17,HOL6326,One of the marquee features of Oracle Applicat...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6326,"{'alpha0': 'HOL', 'numeric1': '6326'}",hol6326,153134816117500185FU,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0


#Resources

Dataquest Data Science Blog — Jupyter Notebook for Beginners: A Tutorial https://www.dataquest.io/blog/jupyter-notebook-tutorial/

The Pandas DataFrame – loading, editing, and viewing data in Python - by Shane Lynn https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/